In [1]:
from lionagi.core.generic.node import BaseNode

In [2]:
a = BaseNode(content="Hello World")

In [3]:
# basic property, cannot be modified
a.class_name

'BaseNode'

### Base Attributes

In [4]:
# id_ is a 32 char string, unique across all base components
# timestamp is a ISO formatted string with timezone set to utc
# content / metadata are zero initialized

print("id: ", a.id_)
print("timestamp: ", a.timestamp)
print("content: ", a.content)
print("metadata: ", a.metadata)

id:  1d8148541d7c2886cfbf79a044802b3f
timestamp:  2024-04-26T16:39:41.759395+00:00
content:  Hello World
metadata:  {}


### Type Conversion 

In [5]:
print("json_str: \n\n", a.to_json_str())

json_str: 

 {"id_": "1d8148541d7c2886cfbf79a044802b3f", "timestamp": "2024-04-26T16:39:41.759395+00:00", "content": "Hello World", "metadata": {}}


In [6]:
print("dict: \n\n", a.to_dict())

dict: 

 {'id_': '1d8148541d7c2886cfbf79a044802b3f', 'timestamp': '2024-04-26T16:39:41.759395+00:00', 'content': 'Hello World', 'metadata': {}}


In [7]:
print("xml: \n\n", a.to_xml())

xml: 

 <BaseNode><id_>1d8148541d7c2886cfbf79a044802b3f</id_><timestamp>2024-04-26T16:39:41.759395+00:00</timestamp><content>Hello World</content><metadata /></BaseNode>


In [8]:
print("pd.Series: \n\n", a.to_pd_series())

pd.Series: 

 id_          1d8148541d7c2886cfbf79a044802b3f
timestamp    2024-04-26T16:39:41.759395+00:00
content                           Hello World
metadata                                   {}
dtype: object


### Field Management

In [9]:
### field annotations, the typing information for each field

print("field annotations: \n\n", a._field_annotations)

field annotations: 

 {'id_': ['str'], 'timestamp': ['str'], 'extra_fields': ['dict'], 'content': ['typing.any', 'none'], 'metadata': ['dict']}


In [10]:
# a field of base component can carry plenty information,
# and we can set a field after the object is created

# there are a few ways of setting a new field

a._add_field(
    field_name="welcome", annotation=str, default="new value", value="hello world again"
)

In [11]:
# we can check the field annotations again

a._field_annotations

{'id_': ['str'],
 'timestamp': ['str'],
 'extra_fields': ['dict'],
 'content': ['typing.any', 'none'],
 'metadata': ['dict'],
 'welcome': ['str']}

In [12]:
# to access the attributes of a partcular field, we can use the following
# for example, we added default value to the field, let's check it

print("default value: \t", a._get_field_attr("welcome", "default", None))
print("current value: \t", getattr(a, "welcome", None))

default value: 	 new value
current value: 	 hello world again


In [13]:
# you can also add a field by passing in a field object

from pydantic import Field

field = Field(
    default="a second hello world",
    description="a second welcome message from the base component",
)

a._add_field(field_name="welcome2", annotation=str, field=field)

In [14]:
a.to_dict()

{'id_': '1d8148541d7c2886cfbf79a044802b3f',
 'timestamp': '2024-04-26T16:39:41.759395+00:00',
 'content': 'Hello World',
 'metadata': {},
 'welcome': 'hello world again',
 'welcome2': 'a second hello world'}

In [15]:
a.extra_fields

{'welcome': FieldInfo(annotation=str, required=False, default='new value'),
 'welcome2': FieldInfo(annotation=str, required=False, default='a second hello world', description='a second welcome message from the base component')}

In [16]:
a._field_annotations

{'id_': ['str'],
 'timestamp': ['str'],
 'extra_fields': ['dict'],
 'content': ['typing.any', 'none'],
 'metadata': ['dict'],
 'welcome': ['str'],
 'welcome2': ['str']}

In [17]:
# notice we didn't set value when adding the field, so it is initialize with default value

print("default value: \t", a._get_field_attr("welcome2", "default", None))
print("current value: \t", getattr(a, "welcome2", None))

default value: 	 a second hello world
current value: 	 a second hello world


### Creation Routine `from_obj`

you can create a base node with various data types

from `dict`

In [18]:
# from_obj: dict

dict_obj = {"a": 1, "b": 2}

b = BaseNode.from_obj(dict_obj)
b.to_dict()

{'id_': '58334f7a2b6f00addd29bc5faf97437a',
 'timestamp': '2024-04-26T16:39:41.829727+00:00',
 'content': None,
 'metadata': {},
 'a': 1,
 'b': 2}

from json formatted `str`

In [19]:
# from_obj: json_str

json_str_obj = '{"a": 1, "b": 2}'

a = BaseNode.from_obj(json_str_obj)
a.to_dict()

{'id_': 'ffba7b9f328e9fa15ded7b3d49d741e4',
 'timestamp': '2024-04-26T16:39:41.838489+00:00',
 'content': None,
 'metadata': {},
 'a': 1,
 'b': 2}

In [20]:
# the json str also supports fuzzy parse
# here is an incorrectly formated json_str for example
json_str_obj = '{"name": "John", "age": 30, "city": ["New York", "DC", "LA"]'

a = BaseNode.from_obj(json_str_obj, fuzzy_parse=True)
a.to_dict()

{'id_': '6c637138429704c6f2abfbf1c1c816fc',
 'timestamp': '2024-04-26T16:39:41.846332+00:00',
 'content': None,
 'metadata': {},
 'name': 'John',
 'age': 30,
 'city': ['New York', 'DC', 'LA']}

from `pd.Series`

In [21]:
# you can create a component object from pandas series / dataframe
import pandas as pd

series_obj = pd.Series({"a": 1, "b": 2})

a = BaseNode.from_obj(series_obj)
a.to_dict()

{'id_': '080256094615d0f00791df84c9143066',
 'timestamp': '2024-04-26T16:39:41.851304+00:00',
 'content': None,
 'metadata': {},
 'a': 1,
 'b': 2}

In [22]:
series_obj.index

Index(['a', 'b'], dtype='object')

In [23]:
df = pd.DataFrame({"a": [1, 2], "b": [3, 4]}, index=["row1", "row2"])
df

,a,b
row1,1,3
row2,2,4


In [24]:
# when you create component object from dataframe, the output will be a list of component objects
# the index is ignored
a = BaseNode.from_obj(df)
type(a)

list

In [25]:
a

[BaseNode(id_='af096cdd5714a4461ef4ff5d5962f77e', timestamp='2024-04-26T16:39:41.868919+00:00', extra_fields={}, content=None, metadata={'df_index': 'row1'}, a=1, b=3),
 BaseNode(id_='5439e1a3730bd20ee9c40975c2ba0427', timestamp='2024-04-26T16:39:41.868973+00:00', extra_fields={}, content=None, metadata={'df_index': 'row2'}, a=2, b=4)]

### from pydantic base model

this allows flexible integrations with other packages such as `llama-index` or `langchain`

In [26]:
from llama_index.core.schema import TextNode

llama_node = TextNode(content="Hello World")

type(llama_node)

llama_index.core.schema.TextNode

In [27]:
from pydantic import BaseModel

isinstance(llama_node, BaseModel)

False

In [28]:
lion_node = BaseNode.from_obj(llama_node)

In [29]:
type(llama_node)

llama_index.core.schema.TextNode

In [30]:
lion_node

BaseNode(id_='035805dc-4969-4c98-8078-deca03330242', timestamp='2024-04-26T16:39:42.505372+00:00', extra_fields={}, content='', metadata={}, embedding=None, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', class_name='TextNode')

In [31]:
from langchain.schema import Document

lc_doc = Document(page_content="Hello World")

In [32]:
type(lc_doc)

langchain_core.documents.base.Document

In [33]:
lion_node = BaseNode.from_obj(lc_doc)
lion_node.metadata

{}

In [34]:
lion_node.meta_insert("author", "John Doe")
lion_node.metadata

{'author': 'John Doe'}

In [35]:
lion_node.meta_change_key("author", "writer")
lion_node.metadata

{'writer': 'John Doe'}

In [36]:
new_meta = {"published": "2024-01-03"}

lion_node.meta_merge(new_meta)
lion_node.metadata

{'writer': 'John Doe', 'published': '2024-01-03'}

In [37]:
print(lion_node)

BaseNode({"id_": "8c4490bda7b97a9529e1c6253dfe8244", "timestamp": "2024-04-26T16:39:42.657083+00:00", "content": "Hello World", "metadata": {"writer": "John Doe", "published": "2024-01-03"}, "lc_id": ["langchain", "schema", "document", "Document"]})
